In [10]:
import numpy as np
import pandas as pd

# Load data
# Load the dataset, selecting only the specified columns
cols = ['company_id', 'title', 'description', 'max_salary', 'med_salary', 'min_salary', 'pay_period', 'formatted_work_type', 'location', 'work_type', 'formatted_experience_level']
data = pd.read_csv("./raw_data/job_postings.csv", usecols=cols)

company_industries = pd.read_csv("./raw_data/company_details/company_industries.csv")
companies = pd.read_csv("./raw_data/company_details/companies.csv")


# Filter out entries lacking salary data
data = data.dropna(how="all", subset=['max_salary','med_salary', 'min_salary'])

In [11]:
from data_processing import standardize_salary
  
"""Standardize salary"""  
data['standardized_annual_salary'] = data.apply(standardize_salary, axis=1)


In [12]:
"""Label rows by salary interval"""  
bins = [0] + list(range(10000, 160000, 10000)) + [float('inf')]
labels = ['10K-'] + [f"{i}K - {i + 10}K" for i in range(10, 150, 10)] + ['160K+']

data['salary_level'] = pd.cut(data['standardized_annual_salary'], bins=bins, labels=labels, right=False)

In [13]:
data

,company_id,title,description,max_salary,med_salary,min_salary,pay_period,formatted_work_type,location,formatted_experience_level,work_type,standardized_annual_salary,salary_level
0,553718.0,Hearing Care Provider,Overview\n\nHearingLife is a national hearing ...,NaN,5250.00,NaN,MONTHLY,Full-time,"Little River, SC",Entry level,FULL_TIME,63000.0,60K - 70K
3,18213359.0,Cook,descriptionTitle\n\n Looking for a great oppor...,NaN,22.27,NaN,HOURLY,Full-time,"Aliso Viejo, CA",Entry level,FULL_TIME,42758.4,40K - 50K
4,437225.0,Principal Cloud Security Architect (Remote),"Job Summary\nAt iHerb, we are on a mission to ...",275834.0,NaN,205956.0,YEARLY,Full-time,United States,Mid-Senior level,FULL_TIME,240895.0,160K+
8,18213359.0,Dishwasher,"descriptionTitle\n\n $2,000 Sign-on Bonus Guar...",NaN,19.30,NaN,HOURLY,Full-time,"Aliso Viejo, CA",Entry level,FULL_TIME,37056.0,30K - 40K
11,19181907.0,Insights Analyst - Auto Industry,Who We Are\n\nEscalent is an award-winning dat...,64000.0,NaN,58000.0,YEARLY,Full-time,United States,Entry level,FULL_TIME,61000.0,60K - 70K
...,...,...,...,...,...,...,...,...,...,...,...,...,...
33231,NaN,Mental Health Practitioners,Gail M. Yost and Associates is hiring full tim...,NaN,100000.00,NaN,YEARLY,Full-time,"Minneapolis, MN",NaN,FULL_TIME,100000.0,100K - 110K
33233,61469.0,Registered Nurse (RN) Vaccinator,United Staffing Solutions is partnering with o...,50.0,NaN,50.0,HOURLY,Part-time,"Muskegon, MI",NaN,PART_TIME,96000.0,90K - 100K
33236,3894635.0,Office Associate,Provide clerical and administrative support to...,42000.0,NaN,37000.0,YEARLY,Full-time,"Albany, GA",NaN,FULL_TIME,39500.0,30K - 40K
33244,NaN,Licensed Insurance Agent,While many industries were hurt by the last fe...,52000.0,NaN,45760.0,YEARLY,Full-time,"Chico, CA",NaN,FULL_TIME,48880.0,40K - 50K


In [15]:
from data_processing import clean_description
import spacy

"""Clean descriptions"""  
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
data['cleaned_description'] = data['description'].apply(lambda x: clean_description(x, nlp))


In [17]:
from data_processing import clean_title

"""Clean title"""  
data['cleaned_title'] = data['title'].apply(lambda x: clean_title(x, nlp))

In [18]:
from data_processing import generate_vocabulary
from data_processing import create_title_emb
 
"""Create embedding for title"""  
vocab = generate_vocabulary(data['cleaned_title'])
word_to_vec = {word: np.eye(len(vocab))[i] for i, word in enumerate(vocab)}
data['title_emb'] = data['cleaned_title'].apply(lambda x: create_title_emb(x, word_to_vec))

In [19]:
data['title_emb']

0        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
3        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
4        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
8        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
11       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...
                               ...                        
33231    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
33233    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
33236    [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...
33244    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
33245    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
Name: title_emb, Length: 13352, dtype: object

In [20]:
# Merge company data
data = data.merge(companies, on='company_id', how='left')
data = data.merge(company_industries, on='company_id', how='left')



final_processed_data = data[[
    'company_id', 'name', 'industry', 'cleaned_title', 'cleaned_description', 'work_type',
    'location', 'formatted_experience_level', 'standardized_annual_salary', 'salary_level', 'title_emb'
]]

final_processed_data.to_csv("../data/processed_job_postings.csv", index=False)

In [21]:
final_processed_data[:15]

,company_id,name,industry,cleaned_title,cleaned_description,work_type,location,formatted_experience_level,standardized_annual_salary,salary_level,title_emb
0,553718.0,HearingLife,Retail,hearing care provider,overview hearinglife is a national hearing car...,FULL_TIME,"Little River, SC",Entry level,63000.0,60K - 70K,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,18213359.0,Episcopal Communities & Services,Non-profit Organization Management,cook,descriptiontitle looking for a great opportuni...,FULL_TIME,"Aliso Viejo, CA",Entry level,42758.4,40K - 50K,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,18213359.0,Episcopal Communities & Services,Non-profit Organizations,cook,descriptiontitle looking for a great opportuni...,FULL_TIME,"Aliso Viejo, CA",Entry level,42758.4,40K - 50K,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,437225.0,"iHerb, LLC",Retail,principal cloud security architect remote,job summary at iherb we are on a mission to ma...,FULL_TIME,United States,Mid-Senior level,240895.0,160K+,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,18213359.0,Episcopal Communities & Services,Non-profit Organization Management,dishwasher,descriptiontitle 2000 signon bonus guaranteed ...,FULL_TIME,"Aliso Viejo, CA",Entry level,37056.0,30K - 40K,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
5,18213359.0,Episcopal Communities & Services,Non-profit Organizations,dishwasher,descriptiontitle 2000 signon bonus guaranteed ...,FULL_TIME,"Aliso Viejo, CA",Entry level,37056.0,30K - 40K,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
6,19181907.0,Escalent,Market Research,insights analyst auto industry,who we are escalent is an awardwinning data an...,FULL_TIME,United States,Entry level,61000.0,60K - 70K,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ..."
7,73013724.0,J. Galt,Financial Services,sales manager,position summary our sales manager has managin...,FULL_TIME,"Coeur d'Alene, ID",Mid-Senior level,237500.0,160K+,"[1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
8,18213359.0,Episcopal Communities & Services,Non-profit Organization Management,custodian janitor,descriptiontitle come work as a custodian at a...,FULL_TIME,"Altadena, CA",Entry level,37056.0,30K - 40K,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
9,18213359.0,Episcopal Communities & Services,Non-profit Organizations,custodian janitor,descriptiontitle come work as a custodian at a...,FULL_TIME,"Altadena, CA",Entry level,37056.0,30K - 40K,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [22]:
import pandas as pd
proceeded_large_data = pd.read_csv("../data/processed_job_postings_large.csv")
proceeded_large_data.isna().sum()

company_id                    12658
name                            385
industry                      11949
cleaned_title                     0
cleaned_description               0
work_type                      7955
location                        208
formatted_experience_level    16923
standardized_annual_salary        0
salary_level                      0
title_emb                         0
dtype: int64

In [23]:
proceeded_large_data.dropna(subset=['formatted_experience_level', 'work_type', 'industry'], inplace=True)
proceeded_large_data.to_csv("../data/processed_job_postings_large_noNA.csv", index=False)


proceeded_data = pd.read_csv("../data/processed_job_postings.csv")
proceeded_data.dropna(subset=['formatted_experience_level', 'work_type', 'industry'], inplace=True)
proceeded_data.to_csv("../data/processed_job_postings_noNA.csv", index=False)